In [ ]:
def reset(o, name):
    from recordclass._dataobject import Factory

    copy_default = o.__options__['copy_default']
    val = o.__defaults__[name]
    if val is not None:
        if type(val) is Factory:
            val = Factory(val)
        elif copy_default:
            if isinstance(val, list):
                val = val[:]
            elif isinstance(val, (dict, set)):
                val = val.copy()
            elif hasattr(o, '__copy__'):
                val = val.__copy__():
            
    setattr(o, name, val)


In [1]:
from recordclass import dataobject, asdict, update

In [ ]:
from recordclass import datatype

class datatype_with_state(datatype):
    def __new__(metatype, typename, bases, ns, *,
                gc=False, fast_new=True, readonly=False, iterable=False,
                deep_dealloc=False, sequence=False, mapping=False,
                use_dict=False, use_weakref=False, hashable=False, 
                immutable_type=False, copy_default=False, match=None):
        
        cls = datatype(typename, bases, ns, 
                gc=gc, fast_new=fast_new, readonly=readonly, iterable=iterable,
                deep_dealloc=deep_dealloc, sequence=sequence, mapping=mapping,
                use_dict=use_dict, use_weakref=use_weakref, hashable=hashable, 
                immutable_type=immutable_type, copy_default=copy_default, match=match)
        
        type.__setattr__(cls, '__fields__', cls.__fields__ + ('__state__',))
        cls.__defaults__['__state__'] = None
        cls.__default_vals__ += (None,) 
        cls.__annotations__['__state__'] = dict
        fields_dict = cls.__options__['fields_dict']
        fields_dict['__state__'] = {'type':dict, 'default':None}
        return cls
        

class dataobject_with_state(metaclass=datatype_with_state):

    def save_state(self):
        self.__state__ =  {nm:getattr(self, nm) for nm in self.__fields__ if nm != '__state__'}

    def restore_state(self, state):
        for nm,val in self.__state__.items():
            setattr(self, nm, val)

class Point(dataobject_with_state):
    x:int
    y:int


a = Point(1,2)
state = a.save_state()
print(a)
a.x = 100
a.y = 200
print(a)
a.restore_state(state)
print(a)


In [6]:
from typing import ClassVar

class State:

    __instance_states__: ClassVar[dict] = {}

    @classmethod
    def clear_states(cls):
        cls.__instance_states__ = {}
    
    def del_state(self):
        if id(self) in self.__instance_states__:
            del self.__instance_states__[id(self)]
    
    def save_state(self):
        self.__instance_states__[id(self)] =  tuple(getattr(self, nm) for nm in self.__fields__)

    def restore_state(self):
        _state = self.__instance_states__.get(id(self), ())
        for nm, val in zip(self.__fields__, _state):
            setattr(self, nm, val)

    def __del__(self):
        if id(self) in self.__instance_states__:
            del self.__instance_states__[id(self)]
        

class Point(dataobject, State):
    x:int
    y:int

a = Point(1,2)
print(a)
a.save_state()
print(State.__instance_states__)
print(a)
a.x = 100
a.y = 200
print(a)
a.restore_state()
print(a)
del a
print(State.__instance_states__)


Point(x=1, y=2)
{140413035868368: (1, 2)}
Point(x=1, y=2)
Point(x=100, y=200)
Point(x=1, y=2)
{}
